환경설정

In [6]:
import os
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import tqdm
import jpype
import ast

In [7]:
folder_path = '/Users/jaesolshin/Documents/GitHub/youtube_dashboard'
file_path = os.path.join(folder_path, 'KPOP_comments_merged_preprocessed_with_nouns.csv')
comments_df = pd.read_csv(file_path)

# 키워드 추출

In [8]:

# 불용어 정의
stopwords = ['timecode', '진짜','이번','사람','정말', '뭔데', '그룹', '우리', '생각', '댓글', '느낌','계속', '지금', '최고', '영상', '처음', '축하', '대박', '이건', '당신', '제발', '항상', '아주', '다음', '정도', '모두', '보고', '그냥', '다시', '그것', '역시', '점점', '오늘', '요즘', '가장', '부분', '전부', '제일', '너머', '내용', '뭔가', '모습', '근데', '너무', '아니', '사람들', '같아요', '데리', '여러분', '세상', '자기', '다른']

# 고유명사 리스트 정의
custom_nouns = ['에스파', '조회수', '하이브', '어도어', 'BTS', '아이브', '르세라핌']

In [50]:
import time
from collections import defaultdict
import numpy as np

def extract_keywords_frequency(series_word_list, top_n=30, min_df=1, existing_keywords=None):
    # 시작시간 확인
    start_time = time.time()

    # 기존 키워드가 없으면 빈 리스트로 초기화
    if existing_keywords is None:
        existing_keywords = []

    # 단어별 문서 빈도를 저장할 딕셔너리 (각 단어가 등장한 문서 수)
    doc_frequency = defaultdict(int)

    # 각 문서에서 등장한 단어 추출 및 빈도 계산
    for word_list in series_word_list:
        unique_words_in_doc = list(set(word_list))  # 문서 내에서 중복된 단어는 한 번만 카운트
        for word in unique_words_in_doc:
            doc_frequency[word] += 1

    # min_df 조건을 충족하는 단어들의 빈도를 저장할 딕셔너리
    word_frequencies = defaultdict(int)

    # 각 문서의 단어 빈도 카운트 (min_df 조건에 맞는 단어만 빈도 계산)
    for word_list in series_word_list:
        for word in word_list:
            if doc_frequency[word] >= min_df:
                word_frequencies[word] += 1

    # 상위 빈도 단어 추출
    sorted_words = sorted(word_frequencies.items(), key=lambda item: item[1], reverse=True)
    
    # 기존 키워드 제외하고 새로운 키워드만 추출
    new_keywords_with_weights = [(word, count) for word, count in sorted_words if word not in existing_keywords]
    
    # 상위 n개 키워드만 추출 (중복 제외 후)
    if len(new_keywords_with_weights) >= top_n:
        top_keywords_with_weights = new_keywords_with_weights[:top_n]
    else:
        top_keywords_with_weights = new_keywords_with_weights
        # 추가 키워드 부족 시 기존 데이터와 겹치지 않도록 기존 키워드에서 추가
        additional_keywords_needed = top_n - len(top_keywords_with_weights)
        additional_keywords = np.random.choice(existing_keywords, size=additional_keywords_needed, replace=False).tolist()
        top_keywords_with_weights += [(word, 0) for word in additional_keywords]  # 가중치는 0으로 설정
    
    # 키워드만 추출
    top_keywords = [word for word, _ in top_keywords_with_weights]

    # 가중치를 전체 합으로 표준화
    total_count = sum(count for _, count in top_keywords_with_weights)
    if total_count > 0:
        keywords_with_weights = [(word, count / total_count) for word, count in top_keywords_with_weights]
    else:
        keywords_with_weights = [(word, 0) for word, count in top_keywords_with_weights]

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights

In [10]:
# TF-IDF 기반 키워드 및 상대적 비중 추출 함수
def extract_keywords_tfidf(series_word_list, top_n=100, min_df=1):

    # 시작시간 확인
    start_time = time.time()

    # 리스트 시리즈를 문자열 시리즈로 변형
    word_list_str = series_word_list.apply(lambda words: ' '.join(words))

    # TF-IDF 벡터화 (min_df를 설정해 빈도수가 적은 단어 제거)
    vectorizer = TfidfVectorizer(max_features=1000, min_df=min_df)
    tfidf_matrix = vectorizer.fit_transform(word_list_str)

    # 각 단어의 TF-IDF 점수를 추출
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = np.mean(tfidf_matrix.toarray(), axis=0)  # 전체 문서에서 평균 TF-IDF 추출

    # TF-IDF 점수를 기준으로 상위 키워드 추출
    top_n_indices = tfidf_scores.argsort()[-top_n:][::-1]
    top_keywords = [feature_names[idx] for idx in top_n_indices]

    # 각 단어의 TF-IDF 점수 추출
    weights = [tfidf_scores[idx] for idx in top_n_indices]

    # 가중치를 전체 합으로 표준화
    weights = weights / np.sum(weights)

    # 키워드와 가중치를 묶어서 반환
    keywords_with_weights = list(zip(top_keywords, weights))

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights

In [11]:
from rank_bm25 import BM25Okapi
import time
import numpy as np
import pandas as pd
from collections import defaultdict

# BM25 기반 키워드 및 상대적 비중 추출 함수 (min_df 추가)
def extract_keywords_bm25(series_word_list, top_n=100, min_df=1):
    # 시작시간 확인
    start_time = time.time()

    # 시리즈의 각 리스트를 문서로 간주하여 BM25 모델 생성
    tokenized_corpus = series_word_list.tolist()
    bm25 = BM25Okapi(tokenized_corpus)

    # 각 단어의 출현 빈도를 저장할 딕셔너리
    doc_frequency = defaultdict(int)

    # 문서 내 각 단어의 출현 빈도 계산
    for doc in tokenized_corpus:
        unique_words_in_doc = set(doc)
        for word in unique_words_in_doc:
            doc_frequency[word] += 1

    # 각 단어의 BM25 점수를 저장할 딕셔너리
    word_scores = defaultdict(float)
    total_docs = len(tokenized_corpus)

    # 각 문서에서의 BM25 점수를 계산하여 해당 단어에 대한 점수를 합산
    for doc in tokenized_corpus:
        scores = bm25.get_scores(doc)
        for word, score in zip(doc, scores):
            # 단어의 출현 빈도가 min_df 이상인 경우에만 점수 계산
            if doc_frequency[word] >= min_df:
                word_scores[word] += score

    # 문서 전체에 대한 평균 BM25 점수로 변환
    word_scores = {word: score / total_docs for word, score in word_scores.items()}

    # 상위 BM25 점수를 기준으로 키워드 추출
    sorted_words = sorted(word_scores.items(), key=lambda item: item[1], reverse=True)
    top_keywords = sorted_words[:top_n]

    # 가중치를 전체 합으로 표준화
    total_weight = sum(score for _, score in top_keywords)
    keywords_with_weights = [(word, score / total_weight) for word, score in top_keywords]

    # 종료시간 확인
    end_time = time.time()

    # 소요시간 출력
    print(f"{len(series_word_list)}개 댓글 키워드 추출에 걸린 시간: {end_time - start_time} 초\n")

    return top_keywords, keywords_with_weights

In [51]:
sample_df = comments_df.copy()
#sample_df = comments_df.sample(frac=0.1, random_state=42)
print(len(sample_df), "\n")

# csv 파일로 저장되면서 문자열로 변형된 word_list 컬럼을 다시 리스트로 변환
sample_df['word_list'] = sample_df['word_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 단순 빈도 기반 키워드 추출
print('**단순 빈도 기반 키워드 추출**\n')
keywords1, keywords_with_weights1 = extract_keywords_frequency(sample_df['word_list'], top_n=300, min_df=50)

# 결과 출력
for item in keywords_with_weights1: print(item)
print('\n')

# TF-IDF 기반 키워드 추출
print('**TF-IDF 기반 키워드 추출**\n')
keywords2, keywords_with_weights2 = extract_keywords_tfidf(sample_df['word_list'], top_n=300, min_df=50)

# 결과 출력
for item in keywords_with_weights2: print(item)
print('\n')

648060 

**단순 빈도 기반 키워드 추출**

648060개 댓글 키워드 추출에 걸린 시간: 1.0672390460968018 초

('노래', 0.06267088912923381)
('뉴진스', 0.04044226923645531)
('뮤비', 0.027513441592293745)
('아이브', 0.021102716329881185)
('하이브', 0.02050202693765124)
('사랑', 0.020448338377983023)
('민희진', 0.018436184533026246)
('단월드', 0.014549599670087692)
('컨셉', 0.010167368249052669)
('멤버', 0.0100413168480925)
('응원', 0.009197083699686427)
('에스파', 0.009012675168652106)
('사이비', 0.008673425719154366)
('화이팅', 0.008631408585500976)
('한국', 0.008589391451847587)
('가사', 0.007769279250538831)
('데뷔', 0.00753429454010691)
('아이돌', 0.007198935565947448)
('삭제', 0.007085333686069764)
('하나', 0.007005189894101261)
('중독', 0.006841011834825979)
('르세라핌', 0.006658937588994623)
('대표', 0.006640263307370895)
('여기', 0.0061531757950186354)
('방시혁', 0.006013896777908325)
('음악', 0.006001447256825839)
('앨범', 0.0058388253876858675)
('이제', 0.005617846388471744)
('리즈', 0.005595281631509738)
('파트', 0.005550152117585727)
('아이', 0.005508134983932337)
('컴백', 0.005340

In [17]:
# 출력 옵션 설정: 모든 열을 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.max_rows', None)     # 모든 행 출력

# 데이터프레임 전치 및 출력
compare_result  = pd.DataFrame([keywords1, keywords2]).transpose()
compare_result

,0,1
0,노래,노래
1,뉴진스,뉴진스
2,뮤비,아이브
3,아이브,사랑
4,하이브,뮤비
5,사랑,하이브
6,민희진,민희진
7,단월드,단월드
8,컨셉,화이팅
9,멤버,에스파


# 추가적인 불용어, 교정이 필요한 단어 찾아내기

In [61]:
## keyword_list 초기화 및 설정
keyword_list = pd.DataFrame(columns=['Group', 'Title', 'keyword'])

## 모든 댓글에 대한 키워드 추가
#sample_df['word_list'] = sample_df['word_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
# 단순 빈도 기반 키워드 추출
keywords, _ = extract_keywords_frequency(sample_df['word_list'], top_n=300, min_df=50)
keyword_list['keyword'] = pd.DataFrame(keywords)
keyword_list['Group'] = 'All'
keyword_list['Title'] = 'All'

## 각 그룹에 대한 추가키워드 발견
# Group 열에 속한 그룹들의 리스트
groups = sample_df['Group'].unique()

# 기존 keyword_list에서 추출된 키워드와 중복되지 않도록 새로운 키워드 추출
existing_keywords = keyword_list['keyword'].tolist()

# 그룹별로 새로운 키워드 추출
for group in groups:
    word_list = sample_df[sample_df['Group'] == group]['word_list']
    new_keywords, _ = extract_keywords_frequency(word_list, top_n=30, min_df=50, existing_keywords=existing_keywords)
    
    # 새로운 키워드를 기존 리스트에 추가
    frac = pd.DataFrame()
    frac['keyword'] = new_keywords
    frac['Group'] = group
    frac['Title'] = 'All'
    
    keyword_list = pd.concat([keyword_list, frac])
    keyword_list.drop_duplicates(subset='keyword', keep='first')
    existing_keywords = keyword_list['keyword'].tolist()
## 각 뮤비에 대한 추가 키워드 발견
# Title 열에 속한 그룹들의 리스트
titles = sample_df['Title'].unique()

# 그룹별로 새로운 키워드 추출
for title in titles:
    group = sample_df[sample_df['Title'] == title]['Group'].iloc[0] # 해당 타이틀에 속한 그룹 조회
    word_list = sample_df[sample_df['Title'] == title]['word_list'] # 댓글 시리즈 가져오기
    new_keywords, _ = extract_keywords_frequency(word_list, top_n=10, min_df=50, existing_keywords=existing_keywords)
    
    # 새로운 키워드를 기존 리스트에 추가
    frac = pd.DataFrame()
    frac['keyword'] = new_keywords
    frac['Group'] = group
    frac['Title'] = title
    
    keyword_list = pd.concat([keyword_list, frac])
    keyword_list.drop_duplicates(subset='keyword', keep='first')
    existing_keywords = keyword_list['keyword'].tolist()

# 인덱스 초기화
keyword_list = keyword_list.reset_index(drop=True)

648060개 댓글 키워드 추출에 걸린 시간: 1.0455050468444824 초

23511개 댓글 키워드 추출에 걸린 시간: 0.026419878005981445 초

50699개 댓글 키워드 추출에 걸린 시간: 0.051187992095947266 초

249967개 댓글 키워드 추출에 걸린 시간: 0.5359420776367188 초

45376개 댓글 키워드 추출에 걸린 시간: 0.06146502494812012 초

7425개 댓글 키워드 추출에 걸린 시간: 0.006674289703369141 초

136993개 댓글 키워드 추출에 걸린 시간: 0.16198325157165527 초

33181개 댓글 키워드 추출에 걸린 시간: 0.03946185111999512 초

7846개 댓글 키워드 추출에 걸린 시간: 0.008840084075927734 초

69125개 댓글 키워드 추출에 걸린 시간: 0.0744779109954834 초

23937개 댓글 키워드 추출에 걸린 시간: 0.02672600746154785 초

4339개 댓글 키워드 추출에 걸린 시간: 0.005337953567504883 초

2622개 댓글 키워드 추출에 걸린 시간: 0.002788066864013672 초

551개 댓글 키워드 추출에 걸린 시간: 0.0006890296936035156 초

1442개 댓글 키워드 추출에 걸린 시간: 0.0016980171203613281 초

1406개 댓글 키워드 추출에 걸린 시간: 0.0017058849334716797 초

1416개 댓글 키워드 추출에 걸린 시간: 0.0014960765838623047 초

1916개 댓글 키워드 추출에 걸린 시간: 0.001901865005493164 초

1608개 댓글 키워드 추출에 걸린 시간: 0.0015230178833007812 초

570개 댓글 키워드 추출에 걸린 시간: 0.0007202625274658203 초

3550개 댓글 키워드 추출에 걸린 시간: 0.00431013

In [62]:
keyword_list

,Group,Title,keyword
0,All,All,노래
1,All,All,뉴진스
2,All,All,뮤비
3,All,All,아이브
4,All,All,하이브
5,All,All,사랑
6,All,All,민희진
7,All,All,단월드
8,All,All,컨셉
9,All,All,멤버


In [64]:
keyword_list.to_csv('keyword_list_before_cleaning.csv')

In [142]:
# 특정 단어가 포함된 댓글 조회하기

def keyword_search(hot_word, dataframe=sample_df, likes=False, n_comments=10, random_state=42):

    # 데이터 프레임에서 hot_word를 포함하고 있는 행 필터링
    filtered_df = dataframe[dataframe['comment'].str.contains(hot_word, na=False)]
    print(len(filtered_df))

    # 좋아요 기준 정렬
    if likes:
        filtered_df = filtered_df[['comment','word_list','likes']].sort_values(by='likes', ascending=False)
        result = filtered_df[:n_comments]
    
    # 랜덤으로 조회
    else :
        result = filtered_df[['comment','word_list','likes']].sample(n = n_comments, random_state=42)
    
    return result

In [219]:
keyword_search('레벨', likes=True)

1076


,comment,word_list,likes
309153,뉴진스는 논란이고 뭐고 그냥 행복했으면 좋겠다… 그냥 뉴진스는 어나더레벨임..,"[어나더, 뉴진스, 레벨, 논란]",3929
330901,그냥 매력이 넘사잖아... 방시혁이 키우는 애덜하고 컨셉이 뭐 비슷하고뭐고간에 애들...,"[뉴진스, 컨셉, 시혁, 고간, 방시혁, 매력, 레벨]",2135
418549,예전부터 느꼈지만 MV는 SM이 할리우드급 아니 미국 MV보다 잘 뽑음.. 이번 노...,"[이후, 미국, 지향, 펑키, 예전, 노래, 할리우드, 미래, 넥스트, 레벨]",987
380216,와 이제야 봤는데 블랙맘바 넥스트레벨 세비지랑 컨셉이 엄청 다른데 엄청 익숙하지않으...,"[블랙맘바, 이제야, 컨셉, 세비지, 넥스트, 레벨]",858
390541,넥스트 레벨때도 처음에는 먼가 당황스러웠는데 나도 모르게 중독되어 버렸지...이것도...,"[중독, 당황, 스엠, 넥스트, 레벨]",797
371436,2억 조회수 갑시다이야~~~~~~~~~~~ NEXT LEVEL 2억 조회수 닿을때 ...,"[조회수, 쵝오, 수고, 마이, 넥스트, 레벨]",569
391872,넥스트 레벨 이해했다고 노래 난이도 떡상시키네.... 반복 학습하겠습니다.,"[이해, 난이도, 학습, 노래, 반복, 넥스트, 레벨]",561
422848,ses 드컴투 + 소시 갤럭시 슈퍼노바 플라워파워 + 에펙 피노키오 일렉트릭쇼크 +...,"[에스파, 드컴투, 파워, 피노키오, 트릭, 쇼크, 파의, 정통, 플라워, 소시, ...",547
391145,뭔가 느낌이 저번 넥스트 레벨처럼 초반에는 난해하나 나중에는 대박치는 노래일듯..넥...,"[초반, 박치, 나중, 넥레, 킬링, 파트, 노래, 저번, 넥스트, 레벨]",449
393613,넥스트 레벨보다 더 에스엠피 스러운 곡이 나왔네요 ㅎㅎ 게다가 뭔가 다들 실력이 더...,"[실력, 에스파, 감탄, 파이팅, 감상, 넥스트, 게다가, 레벨, 엠피]",390


In [230]:
group = 'NMIXX'
hot_word = '짱플러'
sample_df[(sample_df['Group'] == group) & (sample_df['comment'].str.contains(hot_word, na=False))][['comment','word_list','likes']][:10]

,comment,word_list,likes


In [227]:
title = '蜚蜚 (FEIFEI)'
hot_word = '스윗'
sample_df[(sample_df['Title'] == title) & (sample_df['comment'].str.contains(hot_word, na=False))][['comment','word_list','likes']][:10]

,comment,word_list,likes
